In [1]:
import sys
import os
import importlib
#root = './drive/My Drive/Colab Notebooks/rl/'
root = './'
sys.path.insert(1, root + 'env')
sys.path.insert(1, root + 'agents')
sys.path.insert(1, root + 'common')

import gym
import pylab
import random
import sarsa_agent, model_based,dreamer,dreamer_sarsa,dreamer_sarsa_filtering,dreamer_sarsa_filtering_curio,dreamer_light
import random_agent
import numpy as np
from collections import deque
import tensorflow as tf
import tensorflow.keras as keras
from agents import autoencoder_rl_wrapper

import time
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import pandas as pd
import pickle
import copy
from keras import backend as K
K.clear_session()
    
try:
    physical_devices = tf.config.experimental.list_physical_devices('GPU')
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

    physical_devices = tf.config.list_physical_devices('GPU')
    tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
except Exception:
    print('GPU error')

/home/sd/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/home/sd/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy

ModuleNotFoundError: No module named 'clusterized_graph_sa'

In [ ]:
#!sudo apt-get install build-essential zlib1g-dev libsdl2-dev libjpeg-dev \
#nasm tar libbz2-dev libgtk2.0-dev cmake git libfluidsynth-dev libgme-dev \
#libopenal-dev timidity libwildmidi-dev unzip
#!sudo apt-get install libboost-all-dev
#!pip install vizdoom

In [ ]:
#как сделать запись в файл? Перегони в картинку, её перегони в верный формат.

In [ ]:
#(224, 224, 3) - размеры resnet101, resnet 101v2

In [ ]:
# Creates and initializes ViZDoom environment.
def initialize_vizdoom(config_file_path,doom_map="map01"):
    print("Initializing doom...")
    game = DoomGame()
    game.load_config(config_file_path)
    game.set_doom_map(doom_map)
    game.set_screen_format(ScreenFormat.RGB24)
    game.set_screen_resolution(ScreenResolution.RES_200X150)
    game.set_depth_buffer_enabled(True)
    # Enables depth buffer.
    game.set_depth_buffer_enabled(True)
    # Enables labeling of in game objects labeling.
    game.set_labels_buffer_enabled(True)
    # Enables buffer with top down map of the current episode/level.
    game.set_automap_buffer_enabled(True)
    # Enables information about all objects present in the current episode/level.
    #game.set_objects_info_enabled(True)
    # Enables information about all sectors (map layout).
    #game.set_sectors_info_enabled(True)
    # Sets other rendering options (all of these options except crosshair are enabled (set to True) by default)
    game.set_render_hud(True)
    game.set_render_minimal_hud(False)  # If hud is enabled
    game.set_render_crosshair(False)
    game.set_render_weapon(True)
    game.set_render_decals(False)  # Bullet holes and blood on the walls
    game.set_render_particles(False)
    game.set_render_effects_sprites(True)  # Smoke and blood
    game.set_render_messages(False)  # In-game messages
    game.set_render_corpses(True)
    game.set_render_screen_flashes(True)  # Effect upon taking damage or picking up items
    # Adds buttons that will be allowed.
    lst_acts = [Button.ATTACK,Button.USE,Button.MOVE_LEFT,Button.MOVE_RIGHT,Button.MOVE_FORWARD,Button.MOVE_BACKWARD,Button.TURN_LEFT,Button.TURN_RIGHT]#,Button.SELECT_NEXT_WEAPON]
    for action in lst_acts:
        game.add_available_button(action)
    # Adds game variables that will be included in state.
    game.add_available_game_variable(GameVariable.AMMO2)
    game.add_available_game_variable(GameVariable.HEALTH)
    game.add_available_game_variable(GameVariable.FRAGCOUNT)
    game.add_available_game_variable(GameVariable.KILLCOUNT)
    game.add_available_game_variable(GameVariable.ITEMCOUNT)
    game.add_available_game_variable(GameVariable.HITCOUNT)
    # Causes episodes to finish after 200 tics (actions)
    game.set_episode_timeout(1000)
    # Makes episodes start after 10 tics (~after raising the weapon)
    game.set_episode_start_time(10)
    # Makes the window appear (turned on by default)
    game.set_window_visible(True)
    # Turns on the sound. (turned off by default)
    game.set_sound_enabled(False)
    # Sets the living reward (for each move) to -1
    game.set_living_reward(1)
    # Sets ViZDoom mode (PLAYER, ASYNC_PLAYER, SPECTATOR, ASYNC_SPECTATOR, PLAYER mode is default)
    game.set_mode(Mode.PLAYER)
    # Enables engine output to console.
    #game.set_console_enabled(True)
    # Initialize the game. Further configuration won't take any effect from now on.
    time.sleep(1)
    game.init()
    print("Doom initialized.")
    return game

In [ ]:
black_square = np.zeros((224, 224, 3))

In [ ]:
EPISODES=3000
doom=1

print('_____',pd.Timestamp.now())
#здесь весь код от инициализации модели до выдачи scores.
# In case of CartPole-v1, maximum length of episode is 500

#env = jet_table_simple.jet_table_simple_env()
#env = gym.make('Seaquest-ramNoFrameskip-v0')
#env = gym.make('BattleZone-ram-v0')
#env = gym.make('Robotank-ram-v0')
#env = gym.make('Pong-ram-v0')
if doom:
    #basic
    #map02 - фаерболы, немного
    #04 - лостсоул
    #06 - мобы с огнестрелом и с фаерболами #2.1 в среднем от рандома
    #10 - мобы с огнестрелом и фаерболами, перепад высот, деревья. Дальняя перестрелка
    #11 - мобы с огнестрелом, аптечки
    #26 - мобы с огнестрелом, патроны
    #27 - телепорт,оружие, вероятно, мобы. Всё закрыто
    #doom2
    #1 - 2 зомбаря
    #2 - 3 зомбаря
    #3 - импы, зомби
    #5 - двустволка, мегаздоровье, зомби по бокам на выходе из комнатки
    #6 - зомби, импы, ревенант
    #9 - кислота, целей не вижу, но за что-то дают награды, видимо, враги всё же есть поблизости
    #11 - много зомби, кислота
    #13 - прилично врагов, в том числе импы и арахнотрон
    #env = initialize_vizdoom(root + 'doom_files/scenarios/doom2.cfg',doom_map="map11")
    #env = initialize_vizdoom(root + 'doom_files/scenarios/doom2.cfg',doom_map="map05")
    #env = initialize_vizdoom(root + 'doom_files/scenarios/defend_the_center.cfg',doom_map="map01")
    env = initialize_vizdoom(root + 'doom_files/scenarios/basic.cfg',doom_map="map06")
    #[Button.MOVE_LEFT,Button.MOVE_RIGHT,Button.MOVE_FORWARD,Button.MOVE_BACKWARD,Button.TURN_LEFT,Button.TURN_RIGHT,Button.ATTACK,Button.USE ]
    # 0 - не делать ничего
    actions = [[0,0,0,0,0,0,0,0],[1,0,0,0,0,0,0,0], [0,1,0,0,0,0,0,0], [0,0,1,0,0,0,0,0], [0,0,0,1,0,0,0,0], [0,0,0,0,1,0,0,0], [0,0,0,0,0,1,0,0], [0,0,0,0,0,0,1,0],[0,0,0,0,0,0,0,1]]
#env = gym.make('Breakout-ram-v0')
#env = gym.make('SpaceInvaders-ram-v0')
#env = gym.make('MsPacman-ram-v0')
#env=CartPoleEnv9()
#env = aa_gun.AA_gun_simple0_env()
# get size of state and action from environment
if doom:
    #state_size = env.get_screen_size()
    state_size = (env.get_screen_height(),env.get_screen_width(),3)
else:
    state_size = env.observation_space.shape[0]
if doom:
    action_size = len(actions)
else:
    action_size = env.action_space.n

In [ ]:
#np.shape(state) # (150, 200, 3)world model mse

In [ ]:
importlib.reload(autoencoder_rl_wrapper)
#importlib.reload(autoencoder_rl_wrapper2)
#sarsa = a2c.A2CAgent(state_size=30, action_size=action_size)
#sarsa = sarsa_agent.SarsaAgent_RND(state_size=30, action_size=action_size,deque_len=5000)
#sarsa = model_based.ModelBasedAgent(state_size=30, action_size=action_size,deque_len=6000)
#sarsa = dreamer.DreamerAgent(state_size=30, action_size=action_size,deque_len=7000)
#sarsa = dreamer_sarsa_filtering.DreamerAgent(state_size=30, action_size=action_size,deque_len=7000)
#sarsa = dreamer_sarsa_filtering_curio.DreamerAgent(state_size=30, action_size=action_size,deque_len=7000)
sarsa = dreamer_light.DreamerAgent(state_size=30, action_size=action_size,deque_len=7000)
#sarsa = random_agent.randomAgent(state_size=30, action_size=action_size)

agent = autoencoder_rl_wrapper.AutoencoderRLWrapper(state_size, action_size, encoder_layers_count=1, 
                                                    path_to_network='encoder_rl_events.h5', 
                                                    x_path=['./data/x68.pkl'],y_path=['./data/y_pic68.pkl'])
sarsa.parent = agent
sarsa.higher_level = agent
agent.add_deeper_rl(sarsa)
agent.render=True

scores, episodes = [], []
agent.epsilon = 1
t = 0
action = 2
frames = deque(maxlen=3000)
#sarsa.raw_buffer = frames

In [ ]:
#state=5 - на 10м эпизоде всё ещё не обучилась world model
#state=5, sargen делает 1 точку

In [ ]:
stop_train = False
for e in range(EPISODES):
    done = False
    score = 0
    if doom:
        env.new_episode()
        state = env.get_state().screen_buffer
    else:
        state = env.reset()
    state = np.reshape(state, [state_size[0], state_size[1], state_size[2]])/255.
    
    kills = 0
    items = 0
    health = 100
    hits = 0
    explain_mode = 0
    while not done:
        t += 1
        # get action for the current state and go one step in environment
        act_verbose = np.random.rand()<0.003
        action = agent.get_action(state,act_verbose)
        if doom:
            t1 = pd.Timestamp.now()
            reward = env.make_action(actions[action])
            reward = env.get_game_variable(GameVariable.KILLCOUNT)-kills#я считаю как хочу
            reward += (env.get_game_variable(GameVariable.ITEMCOUNT)-items)*0.1
            reward += (env.get_game_variable(GameVariable.HITCOUNT)-hits)*0.15
            reward += (env.get_game_variable(GameVariable.HEALTH)-health)*0.005
            kills = env.get_game_variable(GameVariable.KILLCOUNT)
            items = env.get_game_variable(GameVariable.ITEMCOUNT)
            health = env.get_game_variable(GameVariable.HEALTH)
            hits = env.get_game_variable(GameVariable.HITCOUNT)
            if reward!=0:
                print('reward',reward,flush=True)
            try:
                next_state = env.get_state().screen_buffer
            except Exception:
                print('cannot parse frame')
            frames.append(next_state)
            next_state = np.reshape(next_state, [state_size[0], state_size[1], state_size[2]])
            done = env.is_episode_finished() or env.is_player_dead()
            t2 = pd.Timestamp.now()
            #if env.is_player_dead():
            #    reward = -100
        else:
            next_state, reward, done, info = env.step(action)
        
        next_state = np.reshape(next_state, [state_size[0], state_size[1], state_size[2]])
        agent.append_sample(state, action, reward, next_state, done)
        if not stop_train:
            agent.train_model()
        score += reward
        state = next_state
        
        if doom:
            if env.is_player_dead():
                # Use this to respawn immediately after death, new state will be available.
                env.respawn_player()

        if done:
            # every episode, plot the play time
            scores.append(score)
            episodes.append(e)
            plt.plot(episodes, scores, 'b')
            plt.show()
            #pylab.savefig("./save_graph/aa_gun_dqn.png")

            print("episode:", e, "  score:", score,np.mean(scores), "  epsilon:", sarsa.epsilon, 't:', t, 'KILLS',kills,'HITS',hits,'ITEMS',items,flush=True)
            if not stop_train:
                agent.update_target_model()

In [ ]:
len(sarsa.s)

In [ ]:
reward = np.array(sarsa.r)

In [ ]:
#(root + 'doom_files/scenarios/basic.cfg',doom_map="map06")
#2.0296 - у рандома
#sarsa
#3.05 прямо на первых 20 итерациях, больше не знаю, медлительно работает
#Model based (3 шага по 40 тактов вперёд): 2.1-2.5. Дальше долго работает
#Model based (2 шага по 70 тактов вперёд): 2.23, 2.11
#Dreamer: 2.6
#Dreamer sarsa: 
#Dreamer sarsa filtered: 2.41, 2.23,2.8, 2.4
#Dreamer rnd:2.7, 2.8


#(root + 'doom_files/scenarios/doom2.cfg',doom_map="map05")
#0.9811999999999999 у рандома
#sarsa
#0.64 прямо на первых 100 итерациях, больше не знаю, медлительно работает. Отстой
#Model based (2 шага по 70 тактов вперёд): 1.04
#Model based (2 шага по 70 тактов вперёд): 1.01,0.73
#Dreamer 1.27
#Dreamer sarsa: 1.17
#Dreamer sarsa filtered: 1.21,1.02, 1.3
#Dreamer rnd:1.23


#random,0.9752000000000001,defend_the_center,map01
#1.36 у A2C
#3.8 у sarsa уже на 17 эпизодах
#1.588 у sarsa на 90 эпизодах, и он не очень стабильно работал
#sarsa 1.53
#Model based (2 шага по 70 тактов вперёд): 1.1
#Model based (2 шага по 70 тактов вперёд): 1.25, 1.6
#dreamer-sarsa: 1.01, 1.33, 1.17
#Dreamer sarsa filtered: 1.38
#Dreamer rnd: 1.07, 1.04

#doom2 map30
#sarsa -0.014600000000000161

#doom2 map11
#random 0.15
#dreamer-rnd 0.21
#sarsa -0.05


In [ ]:
#сохранить последний раунд
from PIL import Image
d_sz = len(sarsa.done)
d_where=np.where(sarsa.done)[0]
inverced_index=d_where-d_sz
print(inverced_index)

imgs = [Image.fromarray(frames[i]) for i in range(inverced_index[-2]+1,inverced_index[-1])]
# duration is the number of milliseconds between frames; this is 40 frames per second
imgs[0].save("round.gif", save_all=True, append_images=imgs[1:], duration=1/20, loop=0)